# 01 PCA Macro Factors

Extract factors and interpret loadings.


## Table of Contents
- [Standardize](#standardize)
- [Fit PCA](#fit-pca)
- [Interpret loadings](#interpret-loadings)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Unsupervised notebooks help you understand macro structure:
- latent factors (PCA),
- regimes (clustering),
- anomalies/crises.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Forgetting to standardize features before PCA/clustering.
- Over-interpreting clusters as causal regimes.

## Matching Guide
- `docs/guides/04_unsupervised/01_pca_macro_factors.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/04_unsupervised/01_pca_macro_factors.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Use PCA to extract a small number of macro factors from many indicators.

Why PCA is useful here:
- macro series are correlated
- PCA creates orthogonal (uncorrelated) components
- components can act like "macro factors" (growth, inflation, rates, etc.)



## Primer: pandas Time Series Essentials

### DatetimeIndex
Most time series work in pandas assumes your DataFrame is indexed by time.

```python
import pandas as pd

# idx = pd.to_datetime(df['date'])
# df = df.set_index(idx).sort_index()
# assert isinstance(df.index, pd.DatetimeIndex)
```

If you see weird behavior (resample errors, merges not aligning), check:
- `df.index.dtype`
- `df.index.min(), df.index.max()`
- `df.index.is_monotonic_increasing`

### Resampling
> **Goal:** convert one frequency to another.

Common examples:
- daily -> month-end
- monthly -> quarter-end

```python
# month-end last value
# df_me_last = df.resample('ME').last()

# month-end mean
# df_me_mean = df.resample('ME').mean()

# quarter-end mean
# df_q_mean = df.resample('QE').mean()
```

**Interpretation matters.** For economic series:
- using `.last()` treats the end-of-period value as “the period’s value”
- using `.mean()` treats the period average as “the period’s value”

Neither is universally correct; you should choose based on measurement and use case.

### Alignment and merging
When joining multiple time series, you need to ensure they share:
- the same index type (`DatetimeIndex`)
- the same frequency convention (month-end vs month-start; quarter-end vs quarter-start)

```python
# Example: join two series and inspect missingness
# df = df1.join(df2, how='outer').sort_index()
# print(df.isna().sum())
```

### Lags and rolling windows
> **Lag:** use past values as features.

```python
# lag 1 period
# df['x_lag1'] = df['x'].shift(1)

# rolling mean (past-only)
# df['x_roll12'] = df['x'].rolling(12).mean()
```

### Common gotchas
- `shift(-1)` uses the future (leakage for forecasting).
- `rolling(..., center=True)` uses future values.
- Always `dropna()` after creating lags/rolls to get clean modeling rows.

One more gotcha:
- If you resample daily -> monthly and then create lags, your lag is “one month” (not one day). Lags are measured in the current index frequency.


<a id="standardize"></a>
## Standardize

### Goal
Load a monthly panel and standardize features (mean 0, std 1).

PCA is sensitive to scale: if one variable has larger units, it can dominate the components.



### Your Turn (1): Load panel_monthly.csv (or sample)


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'panel_monthly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

df.head()



### Your Turn (2): Build X and standardize


In [ ]:
from sklearn.preprocessing import StandardScaler

x_cols = df.columns.tolist()
X = df[x_cols].dropna().copy()

sc = StandardScaler().fit(X)
X_s = sc.transform(X)

# TODO: Validate standardization
...



<a id="fit-pca"></a>
## Fit PCA

### Goal
Fit PCA and inspect explained variance.



### Your Turn (1): Fit PCA


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3).fit(X_s)

evr = pd.Series(pca.explained_variance_ratio_, index=[f'PC{i+1}' for i in range(pca.n_components_)])
evr



### Your Turn (2): Scree plot


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot explained variance ratio by component.
...



<a id="interpret-loadings"></a>
## Interpret loadings

### Goal
Interpret what each component represents in economic terms.

Loadings tell you which original variables contribute most to each component.



### Your Turn (1): Build a loadings table


In [ ]:
import pandas as pd

loadings = pd.DataFrame(
    pca.components_.T,
    index=x_cols,
    columns=[f'PC{i+1}' for i in range(pca.n_components_)],
)

# TODO: For each PC, list the top + and top - loadings.
...



### Your Turn (2): Name the components


In [ ]:
# TODO: Write a name/interpretation for PC1 and PC2.
notes = """
PC1: ...
PC2: ...
"""
print(notes)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Confirm your feature matrix is standardized (or justify why not).
# Example:
# assert abs(X_scaled.mean(axis=0)).max() < 1e-6
# assert abs(X_scaled.std(axis=0) - 1).max() < 1e-6
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Standardize</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_pca_macro_factors — Standardize
import pandas as pd
from sklearn.preprocessing import StandardScaler

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True).dropna()
X = StandardScaler().fit_transform(panel)
```

</details>

<details><summary>Solution: Fit PCA</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_pca_macro_factors — Fit PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3).fit(X)
pca.explained_variance_ratio_
```

</details>

<details><summary>Solution: Interpret loadings</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_pca_macro_factors — Interpret loadings
import pandas as pd
loadings = pd.DataFrame(pca.components_.T, index=panel.columns, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
loadings.sort_values('PC1', key=abs, ascending=False).head(10)
```

</details>

